In [38]:
# coding: utf-8
import smtplib
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email.mime.image import MIMEImage
from email.mime.multipart import MIMEMultipart
from email import encoders
from getpass import getpass

In [62]:
class Mailer:
    """ メールを送信するクラス """

    # 初期化
    def __init__(self, addr_to, cc_to, subject, body):
        self.password = getpass("Password: ")  # ← ここにGmailのログインパスワードを追加
        self.addr_from = "mine.mail.444@gmail.com"  # ← ここにメールアドレスを追加
        self.addr_to = addr_to
        self.charset = "ISO-2022-JP"
        self.subject = subject
        self.body = body
        self.CC = ", ".join(cc_to)
        
        # self.msg = MIMEText(self.body.encode(self.charset), 'plain', self.charset)
        self.msg = MIMEMultipart()

    def attach(self, file_path):
        fname = open(file_path, "rb")
        payload = MIMEBase("application", "octet-stream")
        payload.set_payload((fname).read())

        encoders.encode_base64(payload) #encode the attachment
        #add payload header with filename
        payload.add_header('Content-Disposition', 'attachment; filename=  test.pdf')
        self.msg.attach(payload)
        
        
        embedImage = MIMEText('<img src="cid:image1" width="160" height="100"><br>', 'html')
        self.msg.attach(embedImage)
        msgText = MIMEText(self.body)
        self.msg.attach(msgText)
        
        #test images
        fp = open('../data/sample.PNG', 'rb')
        msgImage = MIMEImage(fp.read())
        fp.close()
        msgImage.add_header("Content-ID", "<image1>")
        self.msg.attach(msgImage)

    def send(self):
        # メールの設定
        # msg = MIMEText(self.body.encode(self.charset), 'plain', self.charset)
        self.msg['Subject'] = self.subject
        self.msg['From'] = self.addr_from
        self.msg['To'] = self.addr_to
        self.msg['CC'] = self.CC

        # gmailのsmtp経由で送信
        smtp = smtplib.SMTP('smtp.gmail.com', 587)
        smtp.ehlo()
        smtp.starttls()
        smtp.ehlo()
        smtp.login(self.addr_from, self.password)
        smtp.send_message(self.msg)
        smtp.close()

In [59]:
def create_mail_body(full_name):

    body = \
"""

This is a new test with my email thingy.
do you like it?

Thank you so much.

株式会社○○
{}様

お世話になっております。キカガクの今西です。
こちらのメールはPythonによるGmailのテスト送信になります。

よろしくお願いいたします。

━━━━━━━━━━━━━━━━━━━━━━━
今西 航平（Kohei Imanishi）
株式会社キカガク
取締役副社長
Email: kohei.imanishi@kikagaku.co.jp
URL : https://www.kikagaku.co.jp/
━━━━━━━━━━━━━━━━━━━━━━━━
""".format(full_name)

    return body

In [65]:
addr_to = "tamkayeung.adrian@gmail.com"
CC = ["mine.mail.444@gmail.com", "atam378@aucklanduni.ac.nz"]
subject = "How about this?"
body = create_mail_body("Test")
mailer = Mailer(addr_to, CC, subject, body)
mailer.attach("../data/sample.pdf")

In [66]:
mailer.send()